In [2]:
import numpy as np
import pandas as pd
import xlsxwriter

In [4]:
# All Trucks
AllT = pd.read_excel("../DataFiles/StateHighwayVMT.xlsx", sheetname = 'AVMT-All Vehicles', header = 6) 
# Single Unit Trucks and Buses (class 4-7)
SUTB = pd.read_excel("../DataFiles/StateHighwayVMT.xlsx", sheetname = 'AVMT-Single Unit Trucks & Buses', header = 12) 
# Double Unit Trucks (Class 8-10)
DUT  = pd.read_excel("../DataFiles/StateHighwayVMT.xlsx", sheetname = 'AVMT-Double Unit Trucks', header = 8)
# Triple Unit Trucks (Class 11-13)
TUT  = pd.read_excel("../DataFiles/StateHighwayVMT.xlsx", sheetname = 'AVMT-Triple Unit Trucks', header = 8)
# Total Trucks (Class 4-13)
TTT  = pd.read_excel("../DataFiles/StateHighwayVMT.xlsx", sheetname = 'AVMT-Total Trucks', header = 9)
#Annual Vehicle Miles Summary
AVMT = pd.read_excel("../DataFiles/StateHighwayVMT.xlsx", sheetname = 'AVMT Summary', header = 7)

Factors for converting annual truck miles travelled to ESALs. The factors were obtained from www.pavementinteractive.org/article/trucks-and-buses/

In [5]:
#Factor for Single Unit Trucks and Buses is calculates as:
fA = (.57 + .26 + .42 + .42)/4

#Factor for Double Unit Trucks:
fB = (.30 + 1.20 + .93)/3

#Factor for Triple Unit Trucks:
fC = (.82 + 1.06 + 1.39)/3

In [6]:
SUTB.head(10)

Rural    Urban   Rural.1   Urban.1  Rural.2  Urban.2  \
2014 Adams     1735693    26330   8342085       NaN      NaN    81523   
     Asotin        NaN      NaN       NaN       NaN   650049   558116   
     Benton    1464605  1276797   7800914   6182058  4326140  1729926   
     Chelan     332528   918044       NaN       NaN  3460370   892301   
     Clallam   5914780      NaN       NaN       NaN      NaN   317074   
     Clark      970593   182366   8030819  17572791  2042930  1345096   
     Columbia   293397      NaN       NaN       NaN      NaN      NaN   
     Cowlitz    588423      NaN  20921008   8708841  4161134  1572376   
     Douglas    994788      NaN       NaN       NaN  1511909      NaN   
     Ferry     3101994      NaN       NaN       NaN  1559569      NaN   

               Rural.3   Urban.3  Unnamed: 8  
2014 Adams     8497339    273997    18956967  
     Asotin     373933    324491     1906589  
     Benton        NaN   7328770    30109209  
     Chelan    9504893   3465940    18574078  
     Clallam   9041589   4670052    19943495  
     Clark      623538  17480949    48249081  
     Columbia  2864017       NaN     3157414  
     Cowlitz   1462187   5145831    42559799  
     Douglas   9194279   5782554    17483529  
     Ferry      905180       NaN     5566743

In [7]:
SUTB.columns = [u'Rural Collector',      u'Urban Collector',    u'Rural Interstate',    u'Urban Interstate',
          u'Rural Minor Arterial',    u'Urban Minor Arterial',    u'Rural Principle Arterial',    
                u'Urban Principle Arterial', u'Total']

In [8]:
DUT.columns = [u'Rural Collector',      u'Urban Collector',    u'Rural Interstate',    u'Urban Interstate',
          u'Rural Minor Arterial',    u'Urban Minor Arterial',    u'Rural Principle Arterial',    
                u'Urban Principle Arterial', u'Total']

In [9]:
TUT.columns = [u'Rural Collector',      u'Urban Collector',    u'Rural Interstate',    u'Urban Interstate',
          u'Rural Minor Arterial',    u'Urban Minor Arterial',    u'Rural Principle Arterial',    
                u'Urban Principle Arterial', u'Total']

In [10]:
TTT.columns = [u'Rural Collector',      u'Urban Collector',    u'Rural Interstate',    u'Urban Interstate',
          u'Rural Minor Arterial',    u'Urban Minor Arterial',    u'Rural Principle Arterial',    
                u'Urban Principle Arterial', u'Total']

In [11]:
AllT.columns = [u'Rural Collector',      u'Urban Collector',    u'Rural Interstate',    u'Urban Interstate',
          u'Rural Minor Arterial',    u'Urban Minor Arterial',    u'Rural Principle Arterial',    
                u'Urban Principle Arterial', u'Total']

In [12]:
AVMT = AVMT[['Year', 'Rural', 'Urban', 'Total']]

In [13]:
years = DUT.index.levels[0]
counties = DUT.index.levels[1]

In [14]:
countyNames = list(np.array(counties))

In [15]:
countyNames.remove('Total')

In [16]:
newIndex = []
for year in years[::-1]:
    for county in counties:
        if county != 'Total':
            ind = str(int(year)) + '-' +county
            newIndex.append(ind)
    lastIndex = str(int(year)) + '-Total'
    newIndex.append(lastIndex)
print len(newIndex)

429


In [17]:
def addCountyNames(df):
    i = 0
    for year in years:
        if i == 0:
            df.loc[(i):((1)*39-1), 'County'] = countyNames
            df.loc[(i):((1)*39-1), 'Year'] = year
        else:
            df.loc[(i)*39:((i+1)*39-1), 'County'] = countyNames
            df.loc[(i)*39:((i+1)*39-1), 'Year'] = year
        i+=1

In [18]:
def removeEmptyRows(df):
    newdf = df.dropna(axis=0, how='all', thresh=None, 
                      subset=[u'Rural Collector', u'Urban Collector','Rural Interstate', 'Urban Interstate', 
                              'Rural Minor Arterial','Urban Minor Arterial', 'Rural Principle Arterial'], inplace=False)
    newdf.index = newIndex
    addCountyNames(newdf)
    return newdf

In [19]:
# countyNames

In [20]:
AllT = removeEmptyRows(AllT)
SUTB = removeEmptyRows(SUTB)
DUT  = removeEmptyRows(DUT)
TUT  = removeEmptyRows(TUT)
TTT  = removeEmptyRows(TTT)

/home/guerrero/anaconda/lib/python2.7/site-packages/pandas/core/indexing.py:249: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/guerrero/anaconda/lib/python2.7/site-packages/pandas/core/indexing.py:415: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [21]:
SUTB.head()

,Rural Collector,Urban Collector,Rural Interstate,Urban Interstate,Rural Minor Arterial,Urban Minor Arterial,Rural Principle Arterial,Urban Principle Arterial,Total,County,Year
2014-Adams,1735693,26330,8342085,NaN,NaN,81523,8497339,273997,18956967,Adams,2004
2014-Asotin,NaN,NaN,NaN,NaN,650049,558116,373933,324491,1906589,Asotin,2004
2014-Benton,1464605,1276797,7800914,6182058,4326140,1729926,NaN,7328770,30109209,Benton,2004
2014-Chelan,332528,918044,NaN,NaN,3460370,892301,9504893,3465940,18574078,Chelan,2004
2014-Clallam,5914780,NaN,NaN,NaN,NaN,317074,9041589,4670052,19943495,Clallam,2004


In [22]:
# Replacing all NaNs with zeros in the respective datasets
SUTB = SUTB.fillna(0)
DUT = DUT.fillna(0)
TUT = TUT.fillna(0)
TTT = TTT.fillna(0)

In [23]:
def SUTB2Esals(col):
    return col*fA

def DUT2Esals(col):
    return col*fB

def TUT2Esals(col):
    return col*fC

In [24]:
coldic1 = {1: 'RC', 2:'UC', 3:'RI', 4:'UI', 5:'RMA', 6:'UMA', 7:'RPA', 8:'UPA', 9:'TEsals'}
coldic2 = {'RC': 'Rural Collector', 'UC': 'Urban Collector', 'RI': 'Rural Interstate', 'UI': 'Rural Interstate',
           'RMA': 'Rural Minor Arterial', 'UMA': 'Urban Minor Arterial', 'RPA': 'Rural Principle Arterial', 
           'UPA': 'Urban Principle Arterial', 'TEsals': 'Total'}

In [25]:
def calEsals(df, key):
    if key == 'su':
        fun = SUTB2Esals
    elif key == 'du':
        fun = DUT2Esals
    elif key == 'tu':
        fun = TUT2Esals
    elif key == 'tt':
        fun = TTT2Esals
    else:
        return 'wrong entry)'
    for i in np.arange(9)+1:
        df[coldic1[i]] = df[coldic2[coldic1[i]]].apply(fun)

In [26]:
calEsals(SUTB, 'su')
calEsals(DUT, 'du')
calEsals(TUT, 'tu')
# calEsals(TTT, 'tt')

In [27]:
SUTB.head()

,Rural Collector,Urban Collector,Rural Interstate,Urban Interstate,Rural Minor Arterial,Urban Minor Arterial,Rural Principle Arterial,Urban Principle Arterial,Total,County,Year,RC,UC,RI,UI,RMA,UMA,RPA,UPA,TEsals
2014-Adams,1735693,26330,8342085,0,0,81523,8497339,273997,18956967,Adams,2004,724651.8275,10992.7750,3482820.4875,3482820.4875,0.0000,34035.8525,3547639.0325,114393.7475,7914533.7225
2014-Asotin,0,0,0,0,650049,558116,373933,324491,1906589,Asotin,2004,0.0000,0.0000,0.0000,0.0000,271395.4575,233013.4300,156117.0275,135474.9925,796000.9075
2014-Benton,1464605,1276797,7800914,6182058,4326140,1729926,0,7328770,30109209,Benton,2004,611472.5875,533062.7475,3256881.5950,3256881.5950,1806163.4500,722244.1050,0.0000,3059761.4750,12570594.7575
2014-Chelan,332528,918044,0,0,3460370,892301,9504893,3465940,18574078,Chelan,2004,138830.4400,383283.3700,0.0000,0.0000,1444704.4750,372535.6675,3968292.8275,1447029.9500,7754677.5650
2014-Clallam,5914780,0,0,0,0,317074,9041589,4670052,19943495,Clallam,2004,2469420.6500,0.0000,0.0000,0.0000,0.0000,132378.3950,3774863.4075,1949746.7100,8326409.1625


In [28]:
DUT.head()

,Rural Collector,Urban Collector,Rural Interstate,Urban Interstate,Rural Minor Arterial,Urban Minor Arterial,Rural Principle Arterial,Urban Principle Arterial,Total,County,Year,RC,UC,RI,UI,RMA,UMA,RPA,UPA,TEsals
2014-Adams,2242075,31701,34242956,0,0,96345,29887454,664406,67164937,Adams,2004,1816080.75,25677.81,27736794.36,27736794.36,0.00,78039.45,24208837.74,538168.86,54403598.97
2014-Asotin,0,0,0,0,308435,208942,928252,382127,1827756,Asotin,2004,0.00,0.00,0.00,0.00,249832.35,169243.02,751884.12,309522.87,1480482.36
2014-Benton,668453,947294,27770494,16037885,15794988,1379032,0,6638589,69236735,Benton,2004,541446.93,767308.14,22494100.14,22494100.14,12793940.28,1117015.92,0.00,5377257.09,56081755.35
2014-Chelan,161492,284723,0,0,2014503,378815,12375296,1935579,17150408,Chelan,2004,130808.52,230625.63,0.00,0.00,1631747.43,306840.15,10023989.76,1567818.99,13891830.48
2014-Clallam,2167887,0,0,0,0,303236,4982548,2902211,10355882,Clallam,2004,1755988.47,0.00,0.00,0.00,0.00,245621.16,4035863.88,2350790.91,8388264.42


In [29]:
SumVMT = SUTB.add(DUT, fill_value=None).add(TUT, fill_value=None)

In [30]:
SumVMT.head()

,Rural Collector,Urban Collector,Rural Interstate,Urban Interstate,Rural Minor Arterial,Urban Minor Arterial,Rural Principle Arterial,Urban Principle Arterial,Total,County,Year,RC,UC,RI,UI,RMA,UMA,RPA,UPA,TEsals
2014-Adams,5062378,71989,48584270,0,0,222335,45005676,1114881,100061528,AdamsAdamsAdams,6012,3722957.4775,51884.8050,37758774.4575,37758774.4575,0.0000,160544.3325,34973239.2425,844923.6275,77512322.8525
2014-Asotin,0,0,0,0,1009504,779926,1500377,773332,4063139,AsotinAsotinAsotin,6012,0.0000,0.0000,0.0000,0.0000,576839.6075,416282.5700,1124030.4275,517716.1225,2634868.7275
2014-Benton,2231183,2303332,40416744,25132264,24082500,3300850,0,14889584,112356457,BentonBentonBenton,6012,1259875.7675,1386743.5775,31032397.9750,31032397.9750,18917999.2100,2048422.3050,0.0000,9442243.8150,82833809.2775
2014-Chelan,498591,1227012,0,0,5828104,1317441,23961583,5741615,38574347,ChelanChelanChelan,6012,274621.3500,640336.0500,0.0000,0.0000,3461473.6950,729870.0675,16261002.0475,3385553.5800,24752856.5350
2014-Clallam,8671784,0,0,0,0,666632,15082552,7867363,32288331,ClallamClallamClallam,6012,4867546.6500,0.0000,0.0000,0.0000,0.0000,428490.5350,8964399.6375,4622196.6200,18882633.4425


In [31]:
#print DUT.loc['2014-Adams', 'RC'] + SUTB.loc['2014-Adams', 'RC'] == SumVMT.loc['2014-Adams', 'RC']
# print DUT.loc['2014-Adams', 'UI'] + SUTB.loc['2014-Adams', 'UI'] == SumVMT.loc['2014-Adams', 'UI']
# print len(DUT.index.levels[1]) * len(DUT.index.levels[0]) == len(DUT)

In [32]:
len(counties)

39

In [33]:
# addCountyNames(AllT)

In [34]:
# AllT.head(30)

In [35]:
addCountyNames(SumVMT)

In [36]:
SumVMT.columns

Index([         u'Rural Collector',          u'Urban Collector',
               u'Rural Interstate',         u'Urban Interstate',
           u'Rural Minor Arterial',     u'Urban Minor Arterial',
       u'Rural Principle Arterial', u'Urban Principle Arterial',
                          u'Total',                   u'County',
                           u'Year',                       u'RC',
                             u'UC',                       u'RI',
                             u'UI',                      u'RMA',
                            u'UMA',                      u'RPA',
                            u'UPA',                   u'TEsals'],
      dtype='object')

In [37]:
dfT = SumVMT[[u'County', u'Year', u'RC', u'UC', u'RI', u'UI', u'RMA', u'UMA', u'RPA', u'UPA', u'TEsals']]

In [38]:
dfT = dfT[dfT.loc[:,'Year']>0]

In [39]:
len(SumVMT) - len(dfT) == len(years)

True

In [40]:
dfT.head()

,County,Year,RC,UC,RI,UI,RMA,UMA,RPA,UPA,TEsals
2014-Adams,Adams,2004,3722957.4775,51884.8050,37758774.4575,37758774.4575,0.0000,160544.3325,34973239.2425,844923.6275,77512322.8525
2014-Asotin,Asotin,2004,0.0000,0.0000,0.0000,0.0000,576839.6075,416282.5700,1124030.4275,517716.1225,2634868.7275
2014-Benton,Benton,2004,1259875.7675,1386743.5775,31032397.9750,31032397.9750,18917999.2100,2048422.3050,0.0000,9442243.8150,82833809.2775
2014-Chelan,Chelan,2004,274621.3500,640336.0500,0.0000,0.0000,3461473.6950,729870.0675,16261002.0475,3385553.5800,24752856.5350
2014-Clallam,Clallam,2004,4867546.6500,0.0000,0.0000,0.0000,0.0000,428490.5350,8964399.6375,4622196.6200,18882633.4425


In [41]:
dfT.index = range(len(dfT))

In [42]:
sdf = dfT.sort('County')

In [43]:
sdf.head()

,County,Year,RC,UC,RI,UI,RMA,UMA,RPA,UPA,TEsals
0,Adams,2004,3722957.477500,51884.805000,37758774.457500,37758774.457500,0,160544.332500,34973239.242500,844923.627500,77512322.852500
38,Adams,2005,3702419.788488,50446.021778,35756907.420386,35756907.420386,0,156470.276699,33786317.942303,838911.505310,74291472.954963
76,Adams,2006,3784535.520353,0.000000,37256255.277287,37256255.277287,0,314449.057132,36740009.294105,441649.841966,78536898.990843
114,Adams,2007,3784540.632415,0.000000,38133314.611903,38133314.611903,0,310618.756026,37394721.982992,435848.798239,80059044.781575
152,Adams,2008,3528627.744004,0.000000,37301874.825782,37301874.825782,0,304391.328523,36929125.419969,433088.752094,78497108.070372


In [44]:
sdf.index = range(len(sdf))

In [45]:
idf = sdf[['County', 'Year']]

In [46]:
idf = idf[idf.loc[:,'Year'] > 2004]

In [47]:
idf.head()

,County,Year
1,Adams,2005
2,Adams,2006
3,Adams,2007
4,Adams,2008
5,Adams,2009


In [48]:
def getIncrement1(row):
    county = row['County']
    year   = row['Year']
    a      = list(sdf[(sdf.loc[:,'Year'] == (year-1)) & (sdf.loc[:,'County'] == county)].loc[:,'RC'])[0]
    b      = list(sdf[(sdf.loc[:,'Year'] == year) & (sdf.loc[:,'County'] == county)].loc[:,'RC'])[0]
    return (b-a)/a *100

def getIncrement2(row):
    county = row['County']
    year   = row['Year']
    a      = list(sdf[(sdf.loc[:,'Year'] == (year-1)) & (sdf.loc[:,'County'] == county)].loc[:,'UC'])[0]
    b      = list(sdf[(sdf.loc[:,'Year'] == year) & (sdf.loc[:,'County'] == county)].loc[:,'UC'])[0]
    return (b-a)/a *100

def getIncrement3(row):
    county = row['County']
    year   = row['Year']
    a      = list(sdf[(sdf.loc[:,'Year'] == (year-1)) & (sdf.loc[:,'County'] == county)].loc[:,'UI'])[0]
    b      = list(sdf[(sdf.loc[:,'Year'] == year) & (sdf.loc[:,'County'] == county)].loc[:,'UI'])[0]
    return (b-a)/a *100

def getIncrement4(row):
    county = row['County']
    year   = row['Year']
    a      = list(sdf[(sdf.loc[:,'Year'] == (year-1)) & (sdf.loc[:,'County'] == county)].loc[:,'RI'])[0]
    b      = list(sdf[(sdf.loc[:,'Year'] == year) & (sdf.loc[:,'County'] == county)].loc[:,'RI'])[0]
    return (b-a)/a *100

def getIncrement5(row):
    county = row['County']
    year   = row['Year']
    a      = list(sdf[(sdf.loc[:,'Year'] == (year-1)) & (sdf.loc[:,'County'] == county)].loc[:,'RMA'])[0]
    b      = list(sdf[(sdf.loc[:,'Year'] == year) & (sdf.loc[:,'County'] == county)].loc[:,'RMA'])[0]
    return (b-a)/a *100

def getIncrement6(row):
    county = row['County']
    year   = row['Year']
    a      = list(sdf[(sdf.loc[:,'Year'] == (year-1)) & (sdf.loc[:,'County'] == county)].loc[:,'UMA'])[0]
    b      = list(sdf[(sdf.loc[:,'Year'] == year) & (sdf.loc[:,'County'] == county)].loc[:,'UMA'])[0]
    return (b-a)/a *100

def getIncrement7(row):
    county = row['County']
    year   = row['Year']
    a      = list(sdf[(sdf.loc[:,'Year'] == (year-1)) & (sdf.loc[:,'County'] == county)].loc[:,'RPA'])[0]
    b      = list(sdf[(sdf.loc[:,'Year'] == year) & (sdf.loc[:,'County'] == county)].loc[:,'RPA'])[0]
    return (b-a)/a *100


def getIncrement8(row):
    county = row['County']
    year   = row['Year']
    a      = list(sdf[(sdf.loc[:,'Year'] == (year-1)) & (sdf.loc[:,'County'] == county)].loc[:,'UPA'])[0]
    b      = list(sdf[(sdf.loc[:,'Year'] == year) & (sdf.loc[:,'County'] == county)].loc[:,'UPA'])[0]
    return (b-a)/a *100

def getIncrement9(row):
    county = row['County']
    year   = row['Year']
    a      = list(sdf[(sdf.loc[:,'Year'] == (year-1)) & (sdf.loc[:,'County'] == county)].loc[:,'TEsals'])[0]
    b      = list(sdf[(sdf.loc[:,'Year'] == year) & (sdf.loc[:,'County'] == county)].loc[:,'TEsals'])[0]
    return (b-a)/a *100

In [49]:
idf['RCi'] = idf.apply(getIncrement1, axis = 1)
idf['UCi'] = idf.apply(getIncrement2, axis = 1)
idf['RIi'] = idf.apply(getIncrement4, axis = 1)
idf['UIi'] = idf.apply(getIncrement3, axis = 1)
idf['RMAi'] = idf.apply(getIncrement5, axis = 1)
idf['UMAi'] = idf.apply(getIncrement6, axis = 1)
idf['RPAi'] = idf.apply(getIncrement7, axis = 1)
idf['UPAi'] = idf.apply(getIncrement8, axis = 1)
idf['TEsalsi'] = idf.apply(getIncrement9, axis = 1)

In [50]:
# idf.to_excel("../DataFiles/CountyEsalIncrements.xlsx")

In [51]:
# sdf.to_excel("../DataFiles/CountyEsals.xlsx")

In [52]:
AVMT.head()

,Year,Rural,Urban,Total
0,1980,6598173.2,8491214.5,15089387.6
1,1981,7028297.3,9128681.7,16156978.9
2,1982,7112799.6,9243787.1,16356586.7
3,1983,7471215.6,9789398.8,17260614.4
4,1984,7417125.2,10614739.1,18031864.3


In [53]:
TotalTruck = TTT[TTT.loc[:, 'County'] == 0]

In [54]:
AllVehicles = AllT[AllT.loc[:, 'County'].isnull()]

In [55]:
SingleUT = SUTB[SUTB.loc[:, 'County'] == 0]

In [56]:
DoubleUT = DUT[DUT.loc[:, 'County'] == 0]

In [57]:
TripleUT = TUT[TUT.loc[:, 'County'] == 0]

In [58]:
# TripleUT

In [59]:
# sute

In [60]:
def createDF(df):
    ndf = df[['TEsals']]
    ndf.loc[:,'Rural'] =  np.array(df[['RC']]) + np.array(df[['RI']]) + np.array(df[['RMA']]) + np.array(df[['RPA']])
    ndf.loc[:,'Urban'] =  np.array(df[['UC']]) + np.array(df[['UI']]) + np.array(df[['UMA']]) + np.array(df[['UPA']])
    return ndf

In [61]:
sute = createDF(SingleUT)
dute = createDF(DoubleUT)
tute = createDF(TripleUT)

/home/guerrero/anaconda/lib/python2.7/site-packages/pandas/core/indexing.py:490: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item_labels[indexer[info_axis]]] = value


In [62]:
totalEsals = sute.add(dute).add(tute)

In [63]:
totalEsals

,TEsals,Rural,Urban
2014-Total,2.111570e+09,1.117035e+09,9.221564e+08
2013-Total,2.142351e+09,1.115288e+09,9.210958e+08
2012-Total,2.279157e+09,1.277644e+09,9.953212e+08
2011-Total,2.305823e+09,1.286392e+09,1.004837e+09
2010-Total,2.347753e+09,1.310583e+09,1.024734e+09
2009-Total,2.334538e+09,1.303931e+09,1.018388e+09
2008-Total,2.313218e+09,1.279818e+09,1.016014e+09
2007-Total,2.432127e+09,1.344113e+09,1.086723e+09
2006-Total,2.364252e+09,1.289487e+09,1.049294e+09
2005-Total,2.320898e+09,1.252142e+09,1.031908e+09


In [64]:
tttraffic = TotalTruck[['Total']]

In [65]:
TotalTruck.columns

Index([         u'Rural Collector',          u'Urban Collector',
               u'Rural Interstate',         u'Urban Interstate',
           u'Rural Minor Arterial',     u'Urban Minor Arterial',
       u'Rural Principle Arterial', u'Urban Principle Arterial',
                          u'Total',                   u'County',
                           u'Year'],
      dtype='object')

In [66]:
def getTV(df):    
    ndf = df[['Total']]
    ndf.loc[:,'Rural'] =  np.array(df[['Rural Collector']]) + np.array(df[['Rural Interstate']]) + np.array(df[['Rural Minor Arterial']]) + np.array(df[['Rural Principle Arterial']])
    ndf.loc[:,'Urban'] =  np.array(df[['Urban Collector']]) + np.array(df[['Urban Interstate']]) + np.array(df[['Urban Minor Arterial']]) + np.array(df[['Urban Principle Arterial']])
    return ndf

In [67]:
ttv = getTV(TotalTruck)
alltv = getTV(AllVehicles)
ttv.index = range(len(ttv))
alltv.index = range(len(ttv))
truckPercent = ttv

In [68]:
def getRation(df, item):
    for i in range(len(df)):
        df.loc[i, item] = float(np.array(ttv.loc[i, item]))/float(np.array(alltv.loc[i, item]))

In [69]:
# item = 'Rural'
# float(np.array(ttv.loc[0, item]))

In [70]:
for item in list(ttv.columns):
    getRation(truckPercent,item)

/home/guerrero/anaconda/lib/python2.7/site-packages/pandas/core/indexing.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/home/guerrero/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [71]:
# truckPercent

In [72]:
# Percentage of Truck Miles Travelled from Total Vehicle Miles travelled
pRural = list(truckPercent[['Rural']].mean())[0]
pUrban = list(truckPercent[['Urban']].mean())[0]
pTotal = list(truckPercent[['Total']].mean())[0]

In [73]:
ttv = getTV(TotalTruck)
ttv.index = range(len(ttv))
totalEsals.index = range(len(ttv))

In [74]:
totalEsals. columns = ttv.columns

In [75]:
#this function determines the ration of total esals to the truck traffic
def getRateTE(df, item):
    for i in range(len(df)):
        df.loc[i, item] = float(np.array(totalEsals.loc[i, item]))/float(np.array(ttv.loc[i, item]))

In [76]:
fracTTandESALs = ttv.copy()

In [77]:
for item in list(ttv.columns):
    getRateTE(fracTTandESALs,item)

In [78]:
# Fraction of ESALs from Truck Miles Travelled
fRural = list(fracTTandESALs[['Rural']].mean())[0]
fUrban = list(fracTTandESALs[['Urban']].mean())[0]
fTotal = list(fracTTandESALs[['Total']].mean())[0]

In [79]:
# fracTTandESALs

In [94]:
AVMT.head()

,Year,Rural,Urban,Total,ruralTMT,urbanTMT,totalTMT,rESALs,uESALs,tESALs,pRural,pUrban,pTotal
0,1980,6598173.2,8491214.5,15089387.6,1095738.317765,700176.161462,1691903.176381,739952.237659,419523.945046,1096669.995277,NaN,NaN,NaN
1,1981,7028297.3,9128681.7,16156978.9,1167167.703366,752741.002116,1811607.246514,788188.511643,451019.175155,1174260.642225,0.065188,0.075074,0.070751
2,1982,7112799.6,9243787.1,16356586.7,1181200.740275,762232.466162,1833988.345059,797665.023410,456706.167458,1188767.783991,0.012023,0.012609,0.012354
3,1983,7471215.6,9789398.8,17260614.4,1240721.782387,807223.003824,1935352.785930,837859.591387,483663.108994,1254470.917861,0.050390,0.059025,0.055270
4,1984,7417125.2,10614739.1,18031864.3,1231739.156120,875279.652629,2021829.466773,831793.622626,524440.552393,1310523.996131,-0.007240,0.084310,0.044683


In [81]:
# Converts total vehicle miles travelled to truck miles travelled using the factors obtained above
AVMT[['ruralTMT']] = AVMT[['Rural']] * pRural
AVMT[['urbanTMT']] = AVMT[['Urban']] * pUrban
AVMT[['totalTMT']] = AVMT[['Total']] * pTotal

In [82]:
# Converts truck miles travelled to total esals using the ratios obtained above
AVMT[['rESALs']]  = AVMT[['ruralTMT']] * fRural
AVMT[['uESALs']]  = AVMT[['urbanTMT']] * fUrban
AVMT[['tESALs']]  = AVMT[['totalTMT']] * fTotal

In [83]:
import matplotlib.pyplot as plt
plt.plot(AVMT[['rESALs']], 'ro')
# plt.axis([0, 6, 0, 20])
plt.show()

In [89]:
plt.plot(AVMT[['tESALs']], 'ro')
# plt.axis([0, 6, 0, 20])
plt.show()

In [93]:
for index in range(len(AVMT))[1:]:
    #esal percentage increase
    AVMT.loc[index, 'pRural'] = (AVMT.loc[index, 'rESALs'] - AVMT.loc[(index-1), 'rESALs'])/AVMT.loc[(index-1), 'rESALs']
    AVMT.loc[index, 'pUrban'] = (AVMT.loc[index, 'uESALs'] - AVMT.loc[(index-1), 'uESALs'])/AVMT.loc[(index-1), 'uESALs']
    AVMT.loc[index, 'pTotal'] = (AVMT.loc[index, 'tESALs'] - AVMT.loc[(index-1), 'tESALs'])/AVMT.loc[(index-1), 'tESALs']

In [97]:
import matplotlib.pyplot as plt
plt.plot(AVMT[['Year']], AVMT[['pUrban']],'ro')
# plt.axis([0, 6, 0, 20])
plt.show()

In [98]:
AVMT.to_excel("../DataFiles/EsalIncrements.xlsx")